<a href="https://colab.research.google.com/github/mherbert93/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [x] Continue to clean and explore your data. 
- [x] For the evaluation metric you chose, what score would you get just by guessing?
- [x] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [1]:
!pip install category-encoders

     |████████████████████████████████| 102kB 2.3MB/s 


In [0]:
import pandas as pd

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00397/LasVegasTripAdvisorReviews-Dataset.csv", sep=';')

In [3]:
df.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


In [4]:
df['Score'].value_counts(normalize=True)

5    0.450397
4    0.325397
3    0.142857
2    0.059524
1    0.021825
Name: Score, dtype: float64

In [5]:
from sklearn.metrics import accuracy_score

majority_class = df['Score'].mode()[0]
y_pred = [majority_class] * len(df['Score'])

print("Baseline accuracy is: ", accuracy_score(df['Score'], y_pred))

Baseline accuracy is:  0.4503968253968254


In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['Score'], random_state=1337)

In [7]:
train['Score'].value_counts()

5    181
4    131
3     58
2     24
1      9
Name: Score, dtype: int64

In [0]:
def wrangle(X):

    X = X.copy()

    def country_aggregation(row): #reduce number of country categories to 4. 'USA', 'Canada', 'UK', and all else 'other'.
        if row['User country'] != "USA" and row['User country'] != 'Canada' and row['User country'] != 'UK':
            return "Other"
        else:
            return row['User country']

    def score_aggregation(row):
        if row['Score'] == 5:
            return "Excellent"
        elif row['Score'] == 4 or row['Score'] == 3:
            return "Average"
        else:
            return "Bad"

    X['User country'] = X.apply(country_aggregation, axis=1)
    X['Score'] = X.apply(score_aggregation, axis=1)

    return X

In [0]:
train = wrangle(train)
test = wrangle(test)

In [10]:
majority_class = train['Score'].mode()[0]
y_pred = [majority_class] * len(train['Score'])

print("Train baseline accuracy is: ", accuracy_score(train['Score'], y_pred))

Train baseline accuracy is:  0.46898263027295284


In [11]:
majority_class = test['Score'].mode()[0]
y_pred = [majority_class] * len(test['Score'])

print("Test baseline accuracy is: ", accuracy_score(test['Score'], y_pred))

Test baseline accuracy is:  0.46534653465346537


In [0]:
target = 'Score'

train_features = train.drop([target], axis=1)
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

categorical_features = train_features.select_dtypes(exclude='number').nunique().index.tolist()


features = numeric_features + categorical_features

In [0]:
y_train = train[target]
X_train = train[features]
X_test = test[features]
y_test = test[target]

In [14]:
X_train.shape

(403, 19)

In [17]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from xgboost import XGBClassifier
from sklearn.svm import SVC
#from imblearn.over_sampling import RandomOverSampler

tune = False
forest = False
logistic = False
xgboost = False
adaboost = False
svc = False

forest_distributions = {
    'model__n_estimators': range(250, 500, 50),
    'model__max_depth': range(3, 14),
    'model__max_features': range(2, 14),
    'model__min_samples_leaf': range(2, 4)
}
logistic_distributions = {
    'kbest__k': range(1, 20),
    'model__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

xgboost_distributions = {
    'model__n_estimators': [75, 100, 125, 150, 175],
    'model__max_depth': [6, 7, 8, 9, 10, 11, 12, 13],
    'model__learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.10, 0.12, 0.14, 0.16],
    'model__min_child_leaf':[1, 2, 3],
    'model__min_child_weight': [1, 2, 3, 4],
    'model__colsample_bytree':[0.2, 0.3, 0.4, 0.50, 0.60, 0.70],
    'model__subsample':[0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    'model__gamma':[0],
    'model__scale_pos_weight': [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70,
                                75, 80, 85, 90, 95, 100]
 }

adaboost_distributions = {
    'model__n_estimators': range(20, 100, 5),
    'model__learning_rate': [0.0001, 0.0002, 0.0003, 0.0004,
                             0.0005, 0.0006, 0.0007, 0.0008,
                             0.0009, 0.001, 0.002, 0.003, 0.004, 0.005]
}

svc_distributions = {
    'kbest__k': range(1, 20),
    'model__kernel': ['linear', 'poly', 'rbf'],
    'model__C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'model__gamma': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
                     0.0006, 0.0007, 0.0008, 0.0009, 0.001, 0.002, 0.003, 0.004, 0.005]
}

if tune:

    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', RandomForestClassifier(random_state=1337))])

    logistic_pipeline = Pipeline([('encoder', ce.OneHotEncoder()),
                                ('scaler', StandardScaler()),
                                ('kbest', SelectKBest()),
                                ('model', LogisticRegression(random_state=1337))])

    xgboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', XGBClassifier(seed=1337))])

    adaboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', AdaBoostClassifier(random_state=1337))])

    svc_pipeline = Pipeline([('encoder', ce.OneHotEncoder()),
                             ('scaler', StandardScaler()),
                             ('kbest', SelectKBest()),
                             ('model', SVC(random_state=1337, probability=True))])

    forest_search = GridSearchCV(
        forest_pipeline,
        param_grid=forest_distributions,
        cv=3,
        scoring='neg_log_loss',
        verbose=10,
        n_jobs=15
    )
    logistic_search = GridSearchCV(
        logistic_pipeline,
        param_grid=logistic_distributions,
        cv=3,
        scoring='neg_log_loss',
        verbose=10,
        n_jobs=15
    )
    xgboost_search = RandomizedSearchCV(
        estimator=xgboost_pipeline,
        param_distributions=xgboost_distributions,
        n_iter=10000,
        cv=3,
        scoring='neg_log_loss',
        verbose=10,
        random_state=1337,
        n_jobs=15
    )
    adaboost_search = GridSearchCV(
        adaboost_pipeline,
        param_grid=adaboost_distributions,
        cv=3,
        scoring='neg_log_loss',
        verbose=10,
        n_jobs=15
    )
    svc_search = GridSearchCV(
        svc_pipeline,
        param_grid=svc_distributions,
        cv=3,
        scoring='neg_log_loss',
        verbose=10,
        n_jobs=15
    )

    if forest:
        forest_search.fit(X_train, y_train)
        forest_train_pred = forest_search.predict(X_train)
        forest_test_pred = forest_search.predict(X_test)

    if logistic:
        logistic_search.fit(X_train, y_train)
        logistic_train_pred = logistic_search.predict(X_train)
        logistic_test_pred = logistic_search.predict(X_test)

    if xgboost:
        xgboost_search.fit(X_train, y_train)
        xgboost_train_pred = xgboost_search.predict(X_train)
        xgboost_test_pred = xgboost_search.predict(X_test)

    if adaboost:
        adaboost_search.fit(X_train, y_train)
        adaboost_train_pred = adaboost_search.predict(X_train)
        adaboost_test_pred = adaboost_search.predict(X_test)

    if svc:
        X_train_resampled, y_train_resampled = RandomOverSampler(sampling_strategy='minority').fit_resample(X_train, y_train)
        svc_search.fit(X_train_resampled, y_train_resampled)
        svc_train_pred = svc_search.predict(X_train)
        svc_test_pred = svc_search.predict(X_test)

else:
    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                            ('model', RandomForestClassifier(random_state=1337,
                                                        max_depth=3,
                                                        max_features=13,
                                                        min_samples_leaf=2,
                                                        n_estimators=400))])
    logistic_pipeline = Pipeline([('encoder', ce.OneHotEncoder()),
                                ('scaler', StandardScaler()),
                                ('kbest', SelectKBest(k=9)),
                                ('model', LogisticRegression(random_state=1337, C=0.01))])

    xgboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('model', XGBClassifier(random_state=1337, n_estimators=75, min_child_weight=3,
                                                min_child_leaf=2, max_depth=8, learning_rate=0.05,
                                                gamma=0, subsample=0.5, colsample_bytree=0.3, scale_pos_weight=100))])

    adaboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', AdaBoostClassifier(random_state=1337, n_estimators=70,
                                                        learning_rate=0.0008))])

    svc_pipeline = Pipeline([('encoder', ce.OneHotEncoder()),
                             ('scaler', StandardScaler()),
                             ('kbest', SelectKBest(k=18)),
                             ('model', SVC(random_state=1337, probability=True, C=0.8, gamma=0.005,
                                           kernel='rbf'))])

    X_train_resampled, y_train_resampled = RandomOverSampler(sampling_strategy='not majority').fit_resample(X_train, y_train)

    forest_pipeline.fit(X_train_resampled, y_train_resampled)
    forest_train_pred = forest_pipeline.predict(X_train)
    forest_test_pred = forest_pipeline.predict(X_test)

    logistic_pipeline.fit(X_train_resampled, y_train_resampled)
    logistic_train_pred = logistic_pipeline.predict(X_train)
    logistic_test_pred = logistic_pipeline.predict(X_test)

    xgboost_pipeline.fit(X_train_resampled, y_train_resampled)
    xgboost_train_pred = xgboost_pipeline.predict(X_train)
    xgboost_test_pred = xgboost_pipeline.predict(X_test)

    adaboost_pipeline.fit(X_train_resampled, y_train_resampled)
    adaboost_train_pred = adaboost_pipeline.predict(X_train)
    adaboost_test_pred = adaboost_pipeline.predict(X_test)

    svc_pipeline.fit(X_train_resampled, y_train_resampled)
    svc_train_pred = svc_pipeline.predict(X_train)
    svc_test_pred = svc_pipeline.predict(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


KeyError: ignored

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
if tune:
    target_names = ['Average', 'Bad', 'Excellent']

    if forest:
        print(forest_search.best_params_, '\n')
        print("Best Random Forest CV score: ", forest_search.best_score_, '\n')
        print(classification_report(y_test, forest_test_pred, target_names=target_names), '\n')

        print('\n')

    if logistic:
        print(logistic_search.best_params_, '\n')
        print("Best logistic regression CV score: ", logistic_search.best_score_, '\n')
        print(classification_report(y_test, logistic_test_pred, target_names=target_names), '\n')

        print('\n')

    if xgboost:
        print(xgboost_search.best_params_, '\n')
        print("Best xgboost CV score: ", xgboost_search.best_score_, '\n')
        print(classification_report(y_test, xgboost_test_pred, target_names=target_names), '\n')

        print('\n')

    if adaboost:
        print(adaboost_search.best_params_, '\n')
        print("Best adaboost CV score: ", adaboost_search.best_score_, '\n')
        print(classification_report(y_test, adaboost_test_pred, target_names=target_names), '\n')

        print('\n')

    if svc:
        print(svc_search.best_params_, '\n')
        print("Best SVC CV score: ", svc_search.best_score_, '\n')
        print(classification_report(y_test, svc_test_pred, target_names=target_names), '\n')
else:
    target_names = ['Average', 'Bad', 'Excellent']
    print("Random forest test accuracy score:", accuracy_score(y_test, forest_test_pred), '\n')
    print(classification_report(y_test, forest_test_pred, target_names=target_names))

    print('\n')

    print("Logistic regression test accuracy score:", accuracy_score(y_test, logistic_test_pred), '\n')
    print(classification_report(y_test, logistic_test_pred, target_names=target_names))

    print('\n')

    print("Xgboost test accuracy score:", accuracy_score(y_test, xgboost_test_pred), '\n')
    print(classification_report(y_test, xgboost_test_pred, target_names=target_names))

    print('\n')

    print("Adaboost test accuracy score:", accuracy_score(y_test, adaboost_test_pred), '\n')
    print(classification_report(y_test, adaboost_test_pred, target_names=target_names))

    print('\n')

    print("SVC test accuracy score:", accuracy_score(y_test, svc_test_pred), '\n')
    print(classification_report(y_test, svc_test_pred, target_names=target_names))


In [0]:
# import pandas as pd
#
# # Filenames of your submissions you want to ensemble
# files = ['martin-herbert_random_forest_final.csv', 'martin-herbert_xgboost_final.csv', 'martin-herbert_best_submission_gradiantboost.csv']
#
# target = 'status_group'
# submissions = (pd.read_csv(file)[[target]] for file in files)
# ensemble = pd.concat(submissions, axis='columns')
# majority_vote = ensemble.mode(axis='columns')[0]
#
# sample_submission = pd.read_csv('sample_submission.csv')
# submission = sample_submission.copy()
# submission[target] = majority_vote
# submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)

In [0]:
# from sklearn.linear_model import RidgeCV
# from sklearn.svm import SVR
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestRegressor
# import category_encoders as ce
# from sklearn.pipeline import Pipeline
# from sklearn.feature_selection import SelectKBest
# from sklearn.model_selection import GridSearchCV
#
# lr_distributions = {
#      'model__n_estimators': range(100, 300, 50),
#      'model__max_depth': range(1, 8),
#      'model__max_features': range(2, 12),
#      'model__min_samples_leaf': range(1, 3)
# }
#
# alphas = [0.1, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]
#
# lr_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
#                             ('scaler', StandardScaler()),
#                             ('kbest', SelectKBest()),
#                             ('model', RandomForestRegressor(random_state=1337))])
#
# lr_search = GridSearchCV(
#     lr_pipeline,
#     param_grid=lr_distributions,
#     cv=3,
#     scoring='neg_mean_absolute_error',
#     verbose=10,
#     n_jobs=-1
#     )
#
# lr_search.fit(X_train, y_train)
# lr_train_pred = lr_search.predict(X_train)
# lr_test_pred = lr_search.predict(X_test)

# lr_pipeline.fit(X_train, y_train)
# lr_train_pred = lr_pipeline.predict(X_train)
# lr_test_pred = lr_pipeline.predict(X_test)

In [0]:
# def wrangle(X):
#     X = X.copy()
#
#     def score_aggregation(row):
#         if int(round(row['Score'])) == 5:
#             return "Excellent"
#         elif int(round(row['Score'])) == 4 or int(round(row['Score'])) == 3:
#             return "Average"
#         else:
#             return "Bad"
#
#     if isinstance(X, np.ndarray):
#         lr_test_pred_binned = np.empty(shape=(len(lr_test_pred), 1), dtype=object)
#
#         for i in range(len(lr_test_pred)):
#             if int(round(lr_test_pred[i])) == 5:
#                 lr_test_pred_binned[i] = 'Excellent'
#             elif int(round(lr_test_pred[i])) == 4 or int(round(lr_test_pred[i])) == 3:
#                 lr_test_pred_binned[i] = 'Average'
#             else:
#                 lr_test_pred_binned[i] = 'Bad'
#         return lr_test_pred_binned
#     else:
#         X['Score'] = X.apply(score_aggregation, axis=1)
#         return X

In [0]:
# from sklearn.metrics import classification_report
#
# test = wrangle(test)
#
# y_test = test[target]
# lr_test_pred = wrangle(lr_test_pred)
#
# target_names = ['Average', 'Bad', 'Excellent']
# print(lr_search.best_params_, '\n')
# print("Linear regression test accuracy score:", accuracy_score(y_test, lr_test_pred), '\n')
# print(classification_report(y_test, lr_test_pred, target_names=target_names))

In [0]:
import matplotlib.pyplot as plt

if tune:
    model = forest_search.best_estimator_.named_steps.model
    encoder = forest_search.best_estimator_.named_steps.encoder
else:
    model = forest_pipeline.named_steps.model
    encoder = forest_pipeline.named_steps.encoder


encoded_columns = encoder.transform(X_test).columns
importances = pd.Series(model.feature_importances_, encoded_columns)
plt.figure(figsize=(10,30))
importances.sort_values().plot.barh(color='grey');

In [0]:
import matplotlib.pyplot as plt

import seaborn as sns
plt.figure(figsize=(16,9))
sns.barplot(
    x=train[train['Nr. reviews'] < 200]['Nr. reviews'],
    y=train['Score'],
    color='grey'
);